In [2]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
import gym
import numpy as np

In [5]:
env_id = 'HalfCheetah-v3'  # MuJoCo simulator is needed for this env
env = gym.make(env_id)

running build_ext
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -Ivenv/lib/python3.10/site-packages/mujoco_py -I/home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/mujoco_py -I/home/nikhil/.mujoco/mujoco210/include -I/home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/numpy/core/include -I/home/nikhil/Documents/CE901/venv/include -I/usr/include/python3.10 -c /home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/mujoco_py/cymj.c -o /home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-3.10/home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/mujoco_py/cymj.o -fopenmp -w
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -Iv

In [ ]:
# Run this command to install the necessary packages for MuJoCo to work properly on Linux system.
# If you're using PyCharm in Linux, run it from terminal.
# sudo apt install libosmesa6-dev libgl1-mesa-glx libglfw3 patchelf xvfb ffmpeg

In [6]:
# This function works only for Cartpole and other simple environments.
def evaluate_policy_cp(mdl, num_episodes=100):
    episodes_reward = []
    mdl_env = mdl.get_env()
    for _ in range(num_episodes):
        ts_rewards = []
        done = False
        obv = env.reset()
        while not done:
            action, _states = mdl.predict(obv)
            obv, reward, done, info = mdl_env.step(action)
            ts_rewards.append(reward)
        episodes_reward.append(np.sum(ts_rewards))

    mean_reward = np.mean(episodes_reward)
    print('Mean reward', mean_reward, 'for Episodes:', num_episodes)
    return mean_reward

In [ ]:
# evaluate_policy_cp(model, num_episodes=100)

In [7]:
model = PPO(MlpPolicy, env, verbose=0, device='cpu')

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-0.32 +/- 0.81


In [9]:
model.learn(total_timesteps=10000)

In [10]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/home/nikhil/Documents/CE901/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-4.96 +/- 0.74


In [11]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [12]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_iden, mdl, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_iden: (str)
  :param mdl: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_iden)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = mdl.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [13]:
record_video(env_id, model, video_length=500, prefix='ppo-'+env_id)

Saving video to /home/nikhil/Documents/CE901/videos/ppo-HalfCheetah-v3-step-0-to-step-500.mp4
